In [66]:
# -*- coding: utf-8 -*-
%config IPCompleter.greedy=True

from pathlib import Path
import os
import requests

import import_ipynb
from functions import get_cumulative_data
from functions import get_player_data

import pandas

from _plotly_future_ import v4_subplots

import plotly
import plotly.graph_objs as go
from plotly.tools import make_subplots

# Functions

In [67]:
def player_stats(df, out_dir):
    players3 = df["full_name_code"].values.tolist()
    players4 = df["full_name_id"].values.tolist()

    df = df.set_index(['full_name_id'])

    for i in range(0, len(players3)):
        player3 = players3[i]
        player4 = players4[i]

        val = (df[df.index.str.startswith(player4)])
        
        md_file = out_dir + player3 + '_text.html'
        dirname = os.path.dirname(md_file)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        print("Generating " + md_file + "...")
        with open(md_file, 'w', encoding='utf8') as f:
            f.write('<head><style>' +
                    'body, table, td { font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif; ' +
                    'color: #fefefe; line-height: 1.2; font-size: 12px; } ' +
                    'h1, h2, h3, h4, h5, h6 { margin: 10px 0 5px 0 !important; ' +
                    'font-family: Helvetica, Arial, sans-serif; color: #fff; '
                    'font-weight: bold; line-height: 0.8; } ' +
                    'h1{font-size:20px} h2{font-size:18px} h3{font-size:16px} ' +
                    'h4{font-size:14px} h5{font-size:12px} h6{font-size:12px} ' +
                    'td{ padding: 2px 5px 2px 5px; }</style></head>')
            f.write('<body><h1>' + val['full_name'][0] + '</h1>')
            f.write('<table><tr><td colspan=2><h3>' + str(val['position'][0]) + ', ' +
                    str(val['price'][0]) + ' £</h3></td><td></td></tr>')
            f.write('<tr><td><b>Team: </b>' + str(val['team_name'][0]) + '</td>' +
                    '<td><b>Status:</b> ' + str(val['avail_status'][0]) + '</td></tr>')

            f.write('<tr><td colspan=2><h3>Basic Stats</h3></td><td id="basic_stats" style="display:none;">' + str(
                val['basic_stats'][0]) + '</td><td></td></tr>')
            f.write('<tr><td><b>Minutes played:</b> ' + str(val['minutes'][0]) + '</td>' +
                    '<td><b>Total Points:</b> ' + str(val['total_points'][0]) + '</td></tr>' +
                    '<tr><td><b>Bonus Points:</b> ' + str(val['bonus'][0]) + '</td>' +
                    '<td><b>BPS:</b> ' + str(val['bps'][0]) + '</td></tr>' +
                    '<tr><td><b>Points Per Game:</b> ' + str(val['points_per_game'][0]) + '</td>' +
                    '<td><b>Times in Dream Team:</b> ' + str(val['dreamteam_count'][0]) + '</td></tr>')

            f.write('<tr><td colspan=2><h3>Quality indexes:</h3></td><td id="quality" style="display:none;">' + str(
                val['quality'][0]) + '</td><td></td></tr>')
            f.write('<tr><td><b>ICT index:</b> ' + str(val['ict_index'][0]) + '</td>' +
                    '<td><b>Form:</b> ' + str(val['form'][0]) + '</td></tr>' +
                    '<tr><td><b>Influence:</b> ' + str(val['influence'][0]) + '</td>' +
                    '<td><b>Threat:</b> ' + str(val['threat'][0]) + '</td></tr>' +
                    '<tr><td><b>Creativity:</b> ' + str(val['creativity'][0]) + '</td>' +
                    '<td></td></tr>')

            f.write('<tr><td colspan=2><h3>In-Game Stats</h3></td><td id="in_game_stats" style="display:none;">' + str(
                val['in_game_stats'][0]) + '</td><td></td></tr>')
            f.write('<tr><td><b>Goals Scored:</b> ' + str(val['goals_scored'][0]) + '</td>' +
                    '<td><b>Assists:</b> ' + str(val['assists'][0]) + '</td></tr>' +
                    '<tr><td><b>Goals Conceded:</b> ' + str(val['goals_conceded'][0]) + '</td>' +
                    '<td><b>Clean Sheets:</b> ' + str(val['clean_sheets'][0]) + '</td></tr>' +
                    '<tr><td><b>Saves:</b> ' + str(val['saves'][0]) + '</td>' +
                    '<td><b>Own Goals:</b> ' + str(val['own_goals'][0]) + '</td></tr>' +
                    '<tr><td><b>Penalties Missed:</b> ' + str(val['penalties_missed'][0]) + '</td>' +
                    '<td><b>Penalties Saved:</b> ' + str(val['penalties_saved'][0]) + '</td></tr>' +
                    '<tr><td><b>Yellow Cards:</b> ' + str(val['yellow_cards'][0]) + '</td>' +
                    '<td><b>Red Cards:</b> ' + str(val['red_cards'][0]) + '</td></tr>')

            f.write('<tr><td colspan=2><h3>Popularity</td><td id="popularity" style="display:none;">' + str(
                val['popularity'][0]) + '</td><td></td></tr>')
            f.write('<tr><td><b>Transfers Balance:</b> ' + str(val['transfers_balance'][0]) + '</td>' +
                    '<td><b>GW Transfers Balance:</b> ' + str(val['transfers_balance_event'][0]) + '</td></tr>' +
                    '<tr><td><b>Selected by managers:</b> ' + str(val['selected_by_percent'][0]) + '% </td>' +
                    '<td></td></tr></table></body>')

In [68]:
def player_plots(df, out_dir, gw_cnt):
    players = df["full_name"].values.tolist()
    players3 = df["full_name_code"].values.tolist()
    players4 = df["full_name_id"].values.tolist()
    tools = "hover"
    tooltips = """<table style="padding: 5px; color: #888;">
            <tr style="line-height: 0.8; font-size: 17px; font-weight: bold; padding:10; margin: 0">
                <td colspan=3" style="padding:5px;">@name</td>
            </tr>
            <tr style="line-height: 0.8; font-size: 12px; padding:5px !important; margin: 5px;">
                <td style="font-weight: bold; padding:5px;">Gameweek</td>
                <td style="padding:5px;">@x</td>
            </tr>
            <tr style="line-height: 0.8; font-size: 12px; padding:5px !important; margin: 5px;">
                <td style="font-weight: bold; padding:5px;">Price</td>
                <td style="padding:5px;">@y_value £</td>
            </tr>
            <tr style="line-height: 0.8; font-size: 12px; padding:5px !important; margin: 5px;">
                <td style="font-weight: bold; padding:5px;">Points</td>
                <td style="padding:5px;">@y_pts</td>
            </tr>
            <tr style="line-height: 0.8; font-size: 12px; padding:5px !important; margin: 5px;">
                <td style="font-weight: bold; padding:5px;">Minutes</td>
                <td style="padding:5px;">@y_mins</td>
            </tr>
            <tr style="line-height: 0.8; font-size: 12px; padding:5px !important; margin: 5px;">
                <td style="font-weight: bold; padding:5px;">ICT</td>
                <td style="padding:5px;">@y_ict</td>
            </tr>
        </table>
        """
    layout = go.Layout(
        modebar={'bgcolor': 'rgba(0,0,0,0)'},
        hovermode = 'closest',
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    for i in range(0,len(players)):
        player3 = players3[i]
        player4 = players4[i]
        
        # get plots data
        val = get_player_data(base_path=base_path+"scraper/", player=player4)

        # Create traces
        fig = make_subplots(rows=4, cols=1)
        
        fig.add_trace(go.Scatter(x=val['gw'], y=val['value'] / 10,
                            mode='lines',
                            name='cost',
                            showlegend=False,
                            line=dict(width=4),
                            marker=dict(color = 'white', sizemode = 'area'),
                            hovertemplate = "<b>"+players[i]+"</b><br><br>" +
                                "Cost: £%{y:.2f}</br>"+
                                "Gameweek: %{x}</br>"+
                                "<extra></extra>"),
                     row=1, col=1)
        fig.update_yaxes(title_text="Cost", row=1, col=1)
        
        fig.add_trace(go.Scatter(x=val['gw'], y=val['total_points'],
                            mode='lines',
                            name='points',
                            showlegend=False,
                            line=dict(width=4),        
                            marker=dict(color = 'white', sizemode = 'area'),
                            hovertemplate = "<b>"+players[i]+"</b><br><br>" +
                                "Points: %{y}</br>"+
                                "Gameweek: %{x}</br>"+
                                "<extra></extra>"),
                     row=2, col=1)
        fig.update_yaxes(title_text="Points", row=2, col=1)
        
        fig.add_trace(go.Scatter(x=val['gw'], y=val['ict_index'],
                            mode='lines',
                            name='ict index',
                            showlegend=False,
                            line=dict(width=4),
                            marker=dict(color = 'white', sizemode = 'area'),
                            hovertemplate = "<b>"+players[i]+"</b><br><br>" +
                                "Ict index: %{y}</br>"+
                                "Gameweek: %{x}</br>"+
                                "<extra></extra>"),
                     row=3, col=1)
        fig.update_yaxes(title_text="ICT index", row=3, col=1)
        
        fig.add_trace(go.Scatter(x=val['gw'], y=val['minutes'],
                            mode='lines',
                            name='minutes',
                            showlegend=False,
                            fill='none',
                            line=dict(width=4),
                            marker=dict(color = 'white', sizemode = 'area'),
                            hovertemplate = "<b>"+players[i]+"</b><br><br>" +
                                "Minutes: %{y}</br>"+
                                "Gameweek: %{x}</br>"+
                                "<extra></extra>"),
                     row=4, col=1)
        fig.update_yaxes(title_text="Minutes", row=4, col=1)
        
        fig.update_xaxes(title_text="Gameweek", row=4, col=1)
        
        fig.update_xaxes(color='#eee')
        fig.update_yaxes(color='#eee')
        
        fig.update_scenes(bgcolor='rgba(0,0,0,0)')
        
        fig.layout.update(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
                
        md_file = out_dir + player3 + '.html'
        print("Generating " + md_file + "...")       
        plotly.offline.plot(fig, filename=md_file,config={"displayModeBar": False}, auto_open=False)

In [69]:
def map_code_to_str(row):
    return str("<option value=\"{{ site.baseurl }}/html/player_stats/" + 
               row['full_name_code'] + 
               "\">" +
               row['display_name'] +
               "</option>")

In [70]:
def compare_players(season, curr_gw, in_dir, out_dir):
    df = get_cumulative_data(in_dir, season=season)
    df = df.fillna(0)

    player_stats(df, out_dir)
    player_plots(df, out_dir, curr_gw)

In [71]:
def create_select_options_file(season):
    base_path = "../"    
    options_file = base_path + 'docs/html/select_options.html'

    dirname = os.path.dirname(options_file)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    
    df = get_cumulative_data(base_path=base_path + "scraper/", season=season)
    df = df.fillna(0)    

    df["display_name"] = df["web_name"]

    for i in range(len(df)):
        id = df.iloc[i]
        name = df.iloc[i].web_name
        if len(df[df['web_name']==name]) > 1:
            df.loc[df['full_name_id'] == df.iloc[i].full_name_id, ["display_name"]] = name + ' ' + df.iloc[i].first_name        

    df["display_name_lower"] = df["display_name"].str.lower()
    df.sort_values(by="display_name_lower", inplace=True)

    df["html"] = df.apply(map_code_to_str, axis=1)
    df['html'].to_csv(options_file, index=False,header=False, doublequote=False, quotechar=" ", escapechar=" ")

# Main

In [73]:
URL = "https://fantasy.premierleague.com/api/bootstrap-static"
DATA = requests.get(URL).json()
CURR_GW_OBJS = [x for x in DATA['events'] if x['is_current'] == True]
if len(CURR_GW_OBJS) == 0:
    CURR_GW_OBJS = DATA['events']
CURR_GW = CURR_GW_OBJS[-1]['id']
SEASON = '2018-19'

BASE_PATH = '../'
OUT_DIR = '../docs/html/player_stats/'
IN_DIR = '../scraper/'

In [63]:
create_select_options_file(SEASON)

In [64]:
compare_players(SEASON, CURR_GW, IN_DIR, OUT_DIR)

Generating ../docs/html/player_stats/Petr Cech_11334_text.html...
Generating ../docs/html/player_stats/Bernd Leno_80201_text.html...
Generating ../docs/html/player_stats/Laurent Koscielny_51507_text.html...
Generating ../docs/html/player_stats/Héctor Bellerín_98745_text.html...
Generating ../docs/html/player_stats/Nacho Monreal_38411_text.html...
Generating ../docs/html/player_stats/Rob Holding_156074_text.html...
Generating ../docs/html/player_stats/Shkodran Mustafi_69140_text.html...
Generating ../docs/html/player_stats/Sead Kolasinac_111457_text.html...
Generating ../docs/html/player_stats/Konstantinos Mavropanos_233963_text.html...
Generating ../docs/html/player_stats/Stephan Lichtsteiner_27335_text.html...
Generating ../docs/html/player_stats/Sokratis Papastathopoulos_39476_text.html...
Generating ../docs/html/player_stats/Mesut Özil_37605_text.html...
Generating ../docs/html/player_stats/Aaron Ramsey_41792_text.html...
Generating ../docs/html/player_stats/Alex Iwobi_153133_text.h

Generating ../docs/html/player_stats/Joe Hart_15749_text.html...
Generating ../docs/html/player_stats/Ben Gibson_83312_text.html...
Generating ../docs/html/player_stats/Matej Vydra_81183_text.html...
Generating ../docs/html/player_stats/Dwight McNeil_433154_text.html...
Generating ../docs/html/player_stats/Abd-Al-Ali Morakinyo Olaposi Koiki_432793_text.html...
Generating ../docs/html/player_stats/Josh Benson_217401_text.html...
Generating ../docs/html/player_stats/Peter Crouch_3773_text.html...
Generating ../docs/html/player_stats/Anthony Driscoll-Glennon_195471_text.html...
Generating ../docs/html/player_stats/Harry Arter_48615_text.html...
Generating ../docs/html/player_stats/Neil Etheridge_88734_text.html...
Generating ../docs/html/player_stats/Alex Smithies_45220_text.html...
Generating ../docs/html/player_stats/Sean Morrison_49982_text.html...
Generating ../docs/html/player_stats/Sol Bamba_19523_text.html...
Generating ../docs/html/player_stats/Bruno Ecuele Manga_49382_text.html..

Generating ../docs/html/player_stats/Tom Davies_173807_text.html...
Generating ../docs/html/player_stats/Ademola Lookman_219352_text.html...
Generating ../docs/html/player_stats/Davy Klaassen_109065_text.html...
Generating ../docs/html/player_stats/Gylfi Sigurdsson_55422_text.html...
Generating ../docs/html/player_stats/Nikola Vlasic_180151_text.html...
Generating ../docs/html/player_stats/Beni Baningime_195855_text.html...
Generating ../docs/html/player_stats/Dominic Calvert-Lewin_177815_text.html...
Generating ../docs/html/player_stats/Cenk Tosun_66838_text.html...
Generating ../docs/html/player_stats/Richarlison de Andrade_212319_text.html...
Generating ../docs/html/player_stats/Kevin Mirallas_26901_text.html...
Generating ../docs/html/player_stats/Sandro Ramírez_108438_text.html...
Generating ../docs/html/player_stats/Lucas Digne_101188_text.html...
Generating ../docs/html/player_stats/Yerry Mina_164511_text.html...
Generating ../docs/html/player_stats/Bernard Anício Caldeira Duart

Generating ../docs/html/player_stats/Joel Matip_60914_text.html...
Generating ../docs/html/player_stats/Ragnar Klavan_33871_text.html...
Generating ../docs/html/player_stats/Trent Alexander-Arnold_169187_text.html...
Generating ../docs/html/player_stats/Virgil van Dijk_97032_text.html...
Generating ../docs/html/player_stats/Andrew Robertson_122798_text.html...
Generating ../docs/html/player_stats/Alex Oxlade-Chamberlain_81880_text.html...
Generating ../docs/html/player_stats/Jordan Henderson_56979_text.html...
Generating ../docs/html/player_stats/Adam Lallana_39155_text.html...
Generating ../docs/html/player_stats/Sadio Mané_110979_text.html...
Generating ../docs/html/player_stats/Georginio Wijnaldum_41733_text.html...
Generating ../docs/html/player_stats/Mohamed Salah_118748_text.html...
Generating ../docs/html/player_stats/James Milner_15157_text.html...
Generating ../docs/html/player_stats/Fabio Henrique Tavares_116643_text.html...
Generating ../docs/html/player_stats/Naby Keita_175

Generating ../docs/html/player_stats/Jan Bednarek_171771_text.html...
Generating ../docs/html/player_stats/Wesley Hoedt_167075_text.html...
Generating ../docs/html/player_stats/Steven Davis_17339_text.html...
Generating ../docs/html/player_stats/James Ward-Prowse_101178_text.html...
Generating ../docs/html/player_stats/Oriol Romeu Vidal_78056_text.html...
Generating ../docs/html/player_stats/Pierre-Emile Højbjerg_132015_text.html...
Generating ../docs/html/player_stats/Sofiane Boufal_128198_text.html...
Generating ../docs/html/player_stats/Joshua Sims_153379_text.html...
Generating ../docs/html/player_stats/Nathan Redmond_83283_text.html...
Generating ../docs/html/player_stats/Mario Lemina_151086_text.html...
Generating ../docs/html/player_stats/Stuart Armstrong_91047_text.html...
Generating ../docs/html/player_stats/Mohamed Elyounoussi_96787_text.html...
Generating ../docs/html/player_stats/Shane Long_20452_text.html...
Generating ../docs/html/player_stats/Charlie Austin_78356_text.ht

...

Generating ../docs/html/player_stats/Jayson Molumby_220738.html...
Generating ../docs/html/player_stats/Tom Heaton_21205.html...
Generating ../docs/html/player_stats/Nick Pope_98747.html...
Generating ../docs/html/player_stats/Matthew Lowton_68983.html...
Generating ../docs/html/player_stats/Ben Mee_51927.html...
Generating ../docs/html/player_stats/Stephen Ward_40616.html...
Generating ../docs/html/player_stats/James Tarkowski_17761.html...
Generating ../docs/html/player_stats/Kevin Long_41674.html...
Generating ../docs/html/player_stats/Charlie Taylor_103914.html...
Generating ../docs/html/player_stats/Phil Bardsley_17997.html...
Generating ../docs/html/player_stats/Johann Berg Gudmundsson_60586.html...
Generating ../docs/html/player_stats/Steven Defour_39847.html...
Generating ../docs/html/player_stats/Jeff Hendrick_83314.html...
Generating ../docs/html/player_stats/Robbie Brady_90517.html...
Generating ../docs/html/player_stats/Ashley Westwood_60551.html...
Generating ../docs/html/

Generating ../docs/html/player_stats/Jordan Pickford_111234.html...
Generating ../docs/html/player_stats/Maarten Stekelenburg_10318.html...
Generating ../docs/html/player_stats/Leighton Baines_12745.html...
Generating ../docs/html/player_stats/Seamus Coleman_59949.html...
Generating ../docs/html/player_stats/Phil Jagielka_7645.html...
Generating ../docs/html/player_stats/Ashley Williams_19159.html...
Generating ../docs/html/player_stats/Mason Holgate_194164.html...
Generating ../docs/html/player_stats/Jonjoe Kenny_153673.html...
Generating ../docs/html/player_stats/Michael Keane_106611.html...
Generating ../docs/html/player_stats/Cuco Martina_56192.html...
Generating ../docs/html/player_stats/Theo Walcott_20467.html...
Generating ../docs/html/player_stats/Yannick Bolasie_55452.html...
Generating ../docs/html/player_stats/James McCarthy_50472.html...
Generating ../docs/html/player_stats/Morgan Schneiderlin_42774.html...
Generating ../docs/html/player_stats/Idrissa Gueye_80801.html...
Ge

Generating ../docs/html/player_stats/Rachid Ghezzal_115858.html...
Generating ../docs/html/player_stats/Filip Benkovic_213405.html...
Generating ../docs/html/player_stats/Çaglar Söyüncü_218031.html...
Generating ../docs/html/player_stats/Nampalys Mendy_86881.html...
Generating ../docs/html/player_stats/Josh Knight_201667.html...
Generating ../docs/html/player_stats/Harvey Barnes_201666.html...
Generating ../docs/html/player_stats/Youri Tielemans_166989.html...
Generating ../docs/html/player_stats/Simon Mignolet_66797.html...
Generating ../docs/html/player_stats/Loris Karius_104542.html...
Generating ../docs/html/player_stats/Dejan Lovren_38454.html...
Generating ../docs/html/player_stats/Alberto Moreno_100059.html...
Generating ../docs/html/player_stats/Joseph Gomez_171287.html...
Generating ../docs/html/player_stats/Joel Matip_60914.html...
Generating ../docs/html/player_stats/Ragnar Klavan_33871.html...
Generating ../docs/html/player_stats/Trent Alexander-Arnold_169187.html...
Genera

Generating ../docs/html/player_stats/Danny Ings_84939.html...
Generating ../docs/html/player_stats/Alex McCarthy_58376.html...
Generating ../docs/html/player_stats/Fraser Forster_40383.html...
Generating ../docs/html/player_stats/Ryan Bertrand_40146.html...
Generating ../docs/html/player_stats/Maya Yoshida_80447.html...
Generating ../docs/html/player_stats/Cédric Soares_58822.html...
Generating ../docs/html/player_stats/Jack Stephens_88900.html...
Generating ../docs/html/player_stats/Sam McQueen_153373.html...
Generating ../docs/html/player_stats/Jan Bednarek_171771.html...
Generating ../docs/html/player_stats/Wesley Hoedt_167075.html...
Generating ../docs/html/player_stats/Steven Davis_17339.html...
Generating ../docs/html/player_stats/James Ward-Prowse_101178.html...
Generating ../docs/html/player_stats/Oriol Romeu Vidal_78056.html...
Generating ../docs/html/player_stats/Pierre-Emile Højbjerg_132015.html...
Generating ../docs/html/player_stats/Sofiane Boufal_128198.html...
Generating

Generating ../docs/html/player_stats/Carlos Sánchez_42824.html...
Generating ../docs/html/player_stats/Grady Diangana_179830.html...
Generating ../docs/html/player_stats/Joe Powell_195860.html...
Generating ../docs/html/player_stats/Conor Coventry_222017.html...
Generating ../docs/html/player_stats/Alexandre Nascimento Costa Silva_209925.html...
Generating ../docs/html/player_stats/Samir Nasri_28554.html...
Generating ../docs/html/player_stats/Ben Johnson_222018.html...
Generating ../docs/html/player_stats/Nathan Holland_183487.html...
Generating ../docs/html/player_stats/John Ruddy_19236.html...
Generating ../docs/html/player_stats/Will Norris_168399.html...
Generating ../docs/html/player_stats/Willy Boly_90585.html...
Generating ../docs/html/player_stats/Barry Douglas_57913.html...
Generating ../docs/html/player_stats/Matt Doherty_87835.html...
Generating ../docs/html/player_stats/Romain Saïss_107613.html...
Generating ../docs/html/player_stats/Ryan Bennett_41727.html...
Generating .